In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '954438', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

id     Geographic Area Name  \
0  0500000US01001  Autauga County, Alabama   

   Estimate!!Total!!Civilian noninstitutionalized population  \
0                                              54552           

   Margin of Error!!Total!!Civilian noninstitutionalized population  \
0                                                232                  

  Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination  \
0                                                (X)                                             

  Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination  \
0                                                (X)                                                    

   Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19  \
0                                              13915                                                        

   Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19  \
0                                                166                                                               

   Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years  \
0                                              32484                                                              

   Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years  \
0                                                308                                                                     

   Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over  \
0                                               8153                                                                 

   Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over  \
0                                                161                                                                        

  Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination  \
0                                                (X)                                                                 

  Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination  \
0                                                (X)                                                                        

   Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19  \
0                                              13915                                                                            

   Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19  \
0                                                166                                                                                   

   Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years  \
0                                              32484                                                                                  

   Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years  \
0                                                308                                                                                         

   Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over  \
0                                               8153                                                            

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

LAUS Area Code  FIPS State  FIPS County  \
38627   CN0100100000000            1            1   
38628   CN0100300000000            1            3   
38629   CN0100500000000            1            5   
38630   CN0100700000000            1            7   
38631   CN0100900000000            1            9   

                                       Area Title       Period  \
38627   Autauga County, AL                            Feb-21     
38628   Baldwin County, AL                            Feb-21     
38629   Barbour County, AL                            Feb-21     
38630   Bibb County, AL                               Feb-21     
38631   Blount County, AL                             Feb-21     

      Civilian Labor Force      Employed Unemployed_Level Unemployed_Rate  \
38627              25,563        24,783              780              3.1   
38628              95,716        92,589            3,127              3.3   
38629               8,495         7,972              523              6.2   
38630               8,590         8,280              310              3.6   
38631              24,750        24,208              542              2.2   

        FIPS  
38627  01001  
38628  01003  
38629  01005  
38630  01007  
38631  01009

In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


LAUS Area Code  FIPS State  FIPS County  \
41841   CN7214500000000           72          145   
41842   CN7214700000000           72          147   
41843   CN7214900000000           72          149   
41844   CN7215100000000           72          151   
41845   CN7215300000000           72          153   

                                       Area Title       Period  \
41841   Vega Baja Municipio, PR                       Feb-21     
41842   Vieques Municipio, PR                         Feb-21     
41843   Villalba Municipio, PR                        Feb-21     
41844   Yabucoa Municipio, PR                         Feb-21     
41845   Yauco Municipio, PR                           Feb-21     

      Civilian Labor Force      Employed  Unemployed_Level  Unemployed_Rate  \
41841              12,686        11,429               1257              9.9   
41842               2,475         2,183                292             11.8   
41843               6,835         6,021                814             11.9   
41844               8,240         7,352                888             10.8   
41845               8,956         7,900               1056             11.8   

        FIPS  
41841  72145  
41842  72147  
41843  72149  
41844  72151  
41845  72153

In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(100)

date       county          state     fips  cases  deaths
1293677  2021-05-06      Webster  West Virginia  54101.0    475     7.0
1293678  2021-05-06       Wetzel  West Virginia  54103.0   1311    29.0
1293679  2021-05-06         Wirt  West Virginia  54105.0    416     3.0
1293680  2021-05-06         Wood  West Virginia  54107.0   7720   168.0
1293681  2021-05-06      Wyoming  West Virginia  54109.0   1974    39.0
1293682  2021-05-06        Adams      Wisconsin  55001.0   1914    17.0
1293683  2021-05-06      Ashland      Wisconsin  55003.0   1353    15.0
1293684  2021-05-06       Barron      Wisconsin  55005.0   5904    83.0
1293685  2021-05-06     Bayfield      Wisconsin  55007.0   1261    19.0
1293686  2021-05-06        Brown      Wisconsin  55009.0  35035   286.0
1293687  2021-05-06      Buffalo      Wisconsin  55011.0   1388     7.0
1293688  2021-05-06      Burnett      Wisconsin  55013.0   1504    22.0
1293689  2021-05-06      Calumet      Wisconsin  55015.0   6429    54.0
1293690  2021-05-06     Chippewa      Wisconsin  55017.0   7550    96.0
1293691  2021-05-06        Clark      Wisconsin  55019.0   3645    62.0
1293692  2021-05-06     Columbia      Wisconsin  55021.0   5791    69.0
1293693  2021-05-06     Crawford      Wisconsin  55023.0   1769    17.0
1293694  2021-05-06         Dane      Wisconsin  55025.0  45963   325.0
1293695  2021-05-06        Dodge      Wisconsin  55027.0  12751   185.0
1293696  2021-05-06         Door      Wisconsin  55029.0   2816    28.0
1293697  2021-05-06      Douglas      Wisconsin  55031.0   4800    51.0
1293698  2021-05-06         Dunn      Wisconsin  55033.0   5042    32.0
1293699  2021-05-06   Eau Claire      Wisconsin  55035.0  11853   119.0
1293700  2021-05-06     Florence      Wisconsin  55037.0    501    12.0
1293701  2021-05-06  Fond du Lac      Wisconsin  55039.0  13602   121.0
1293702  2021-05-06       Forest      Wisconsin  55041.0   1038    26.0
1293703  2021-05-06        Grant      Wisconsin  55043.0   5670    90.0
1293704  2021-05-06        Green      Wisconsin  55045.0   3618    24.0
1293705  2021-05-06   Green Lake      Wisconsin  55047.0   2010    22.0
1293706  2021-05-06         Iowa      Wisconsin  55049.0   2105    11.0
...             ...          ...            ...      ...    ...     ...
1293747  2021-05-06     Washburn      Wisconsin  55129.0   1535    21.0
1293748  2021-05-06   Washington      Wisconsin  55131.0  16672   165.0
1293749  2021-05-06     Waukesha      Wisconsin  55133.0  48602   577.0
1293750  2021-05-06      Waupaca      Wisconsin  55135.0   6007   164.0
1293751  2021-05-06     Waushara      Wisconsin  55137.0   2493    37.0
1293752  2021-05-06    Winnebago      Wisconsin  55139.0  21175   218.0
1293753  2021-05-06         Wood      Wisconsin  55141.0   7518    93.0
1293754  2021-05-06       Albany        Wyoming  56001.0   4302    12.0
1293755  2021-05-06     Big Horn        Wyoming  56003.0   1152    32.0
1293756  2021-05-06     Campbell        Wyoming  56005.0   4845    60.0
1293757  2021-05-06       Carbon        Wyoming  56007.0   1583    23.0
1293758  2021-05-06     Converse        Wyoming  56009.0   1001    17.0
1293759  2021-05-06        Crook        Wyoming  56011.0    429    12.0
1293760  2021-05-06      Fremont        Wyoming  56013.0   5179    84.0
1293761  2021-05-06       Goshen        Wyoming  56015.0   1207    24.0
1293762  2021-05-06  Hot Springs        Wyoming  56017.0    369     3.0
1293763  2021-05-06      Johnson        Wyoming  56019.0    718    10.0
1293764  2021-05-06      Laramie        Wyoming  56021.0   9130   113.0
1293765  2021-05-06      Lincoln        Wyoming  56023.0   1403    12.0
1293766  2021-05-06      Natrona        Wyoming  56025.0   7972   136.0
1293767  2021-05-06     Niobrara        Wyoming  56027.0    160     2.0
1293768  2021-05-06         Park        Wyoming  56029.0   2718    30.0
1293769  2021-05-06       Platte        Wyoming  56031.0    624    11.0
1293770  2021-05-06     Sheridan        Wyoming  56033.

Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

date_of_interest  CASE_COUNT  PROBABLE_CASE_COUNT  HOSPITALIZED_COUNT  \
425       04/29/2021        1020                  339                  92   
426       04/30/2021         926                  329                  77   
427       05/01/2021         533                  213                  68   
428       05/02/2021         519                  219                  49   
429       05/03/2021         714                  263                   9   

     DEATH_COUNT  PROBABLE_DEATH_COUNT  CASE_COUNT_7DAY_AVG  \
425           26                     7                 1070   
426           34                     7                 1020   
427           21                    14                  977   
428           22                    10                  946   
429           12                     9                  857   

     ALL_CASE_COUNT_7DAY_AVG  HOSP_COUNT_7DAY_AVG  DEATH_COUNT_7DAY_AVG  \
425                     1433                  113                    30   
426                     1371                  103                    28   
427                     1310                   99                    27   
428                     1268                   90                    26   
429                     1163                   74                    24   

     ALL_DEATH_COUNT_7DAY_AVG  BX_CASE_COUNT  BX_PROBABLE_CASE_COUNT  \
425                        33            148                      65   
426                        32            163                      66   
427                        33             80                      46   
428                        33             67                      48   
429                        32            108                      59   

     BX_HOSPITALIZED_COUNT  BX_DEATH_COUNT  BX_PROBABLE_DEATH_COUNT  \
425                     20               6                        0   
426                     16               6                        0   
427                     12               4                        0   
428                     12               0                        0   
429                      5               2                        0   

     BX_CASE_COUNT_7DAY_AVG  BX_ALL_CASE_COUNT_7DAY_AVG  \
425                     168                         239   
426                     161                         231   
427                     156                         224   
428                     152                         219   
429                     136                         200   

     BX_HOSPITALIZED_COUNT_7DAY_AVG  BX_DEATH_COUNT_7DAY_AVG  \
425                              22                        6   
426                              21                        5   
427                              19                        5   
428                              18                        4   
429                              15                        4   

     BX_ALL_DEATH_COUNT_7DAY_AVG  BK_CASE_COUNT  BK_PROBABLE_CASE_COUNT  \
425                            6            373                     110   
426                            5            315                     104   
427                            5            176                      66   
428                            4            194                      72   
429                            4            255                      76   

     BK_HOSPITALIZED_COUNT  BK_DEATH_COUNT  BK_PROBABLE_DEATH_COUNT  \
425                     32               8                        0   
426                     20              12                        1   
427                     19               6                        2   
428                     14              10                        1   
429                      1               3                        0   

     BK_CASE_COUNT_7DAY_AVG  BK_ALL_CASE_COUNT_7DAY_AVG  \
425                     379                         493   
426                     362                         471   
427                     344                         448 

In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


date     county       state     fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061.0      1     0.0
1  2020-01-22  Snohomish  Washington  53061.0      1     0.0
2  2020-01-23  Snohomish  Washington  53061.0      1     0.0
3  2020-01-24       Cook    Illinois  17031.0      1     0.0
4  2020-01-24  Snohomish  Washington  53061.0      1     0.0

In [32]:
Bronx.tail(5)

date  cases  deaths     state county     fips
425  04/29/2021    148       6  New York  Bronx  36005.0
426  04/30/2021    163       6  New York  Bronx  36005.0
427  05/01/2021     80       4  New York  Bronx  36005.0
428  05/02/2021     67       0  New York  Bronx  36005.0
429  05/03/2021    108       2  New York  Bronx  36005.0

In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

date  cases  deaths county     state     fips
0  02/29/2020      0       0  Kings  New York  36047.0
1  03/01/2020      0       0  Kings  New York  36047.0
2  03/02/2020      0       0  Kings  New York  36047.0
3  03/03/2020      0       0  Kings  New York  36047.0
4  03/04/2020      1       0  Kings  New York  36047.0

In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

date  cases  deaths county     state     fips
425  04/29/2021    373       8  Kings  New York  36047.0
426  04/30/2021    315      12  Kings  New York  36047.0
427  05/01/2021    176       6  Kings  New York  36047.0
428  05/02/2021    194      10  Kings  New York  36047.0
429  05/03/2021    255       3  Kings  New York  36047.0

In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

date  cases  deaths county     state     fips         dt
0   02/29/2020      0       0  Kings  New York  36047.0 2020-02-29
1   03/01/2020      0       0  Kings  New York  36047.0 2020-03-01
2   03/02/2020      0       0  Kings  New York  36047.0 2020-03-02
3   03/03/2020      0       0  Kings  New York  36047.0 2020-03-03
4   03/04/2020      1       0  Kings  New York  36047.0 2020-03-04
5   03/05/2020      3       0  Kings  New York  36047.0 2020-03-05
6   03/06/2020      1       0  Kings  New York  36047.0 2020-03-06
7   03/07/2020      2       0  Kings  New York  36047.0 2020-03-07
8   03/08/2020      5       0  Kings  New York  36047.0 2020-03-08
9   03/09/2020     16       0  Kings  New York  36047.0 2020-03-09
10  03/10/2020     11       0  Kings  New York  36047.0 2020-03-10
11  03/11/2020     31       0  Kings  New York  36047.0 2020-03-11
12  03/12/2020     96       0  Kings  New York  36047.0 2020-03-12
13  03/13/2020    166       0  Kings  New York  36047.0 2020-03-13
14  03/14/2020    163       1  Kings  New York  36047.0 2020-03-14
15  03/15/2020    433       0  Kings  New York  36047.0 2020-03-15
16  03/16/2020    740       1  Kings  New York  36047.0 2020-03-16
17  03/17/2020    783       4  Kings  New York  36047.0 2020-03-17
18  03/18/2020    966       6  Kings  New York  36047.0 2020-03-18
19  03/19/2020   1204       6  Kings  New York  36047.0 2020-03-19
20  03/20/2020   1137      11  Kings  New York  36047.0 2020-03-20
21  03/21/2020    554      12  Kings  New York  36047.0 2020-03-21
22  03/22/2020    756      10  Kings  New York  36047.0 2020-03-22
23  03/23/2020    910      27  Kings  New York  36047.0 2020-03-23
24  03/24/2020   1212      23  Kings  New York  36047.0 2020-03-24
25  03/25/2020   1254      37  Kings  New York  36047.0 2020-03-25
26  03/26/2020   1375      50  Kings  New York  36047.0 2020-03-26
27  03/27/2020   1424      75  Kings  New York  36047.0 2020-03-27
28  03/28/2020    863      78  Kings  New York  36047.0 2020-03-28
29  03/29/2020   1149      89  Kings  New York  36047.0 2020-03-29
30  03/30/2020   1754     106  Kings  New York  36047.0 2020-03-30
31  03/31/2020   1513     126  Kings  New York  36047.0 2020-03-31
32  04/01/2020   1589     135  Kings  New York  36047.0 2020-04-01
33  04/02/2020   1611     171  Kings  New York  36047.0 2020-04-02
34  04/03/2020   1440     143  Kings  New York  36047.0 2020-04-03
35  04/04/2020    921     163  Kings  New York  36047.0 2020-04-04
36  04/05/2020   1141     166  Kings  New York  36047.0 2020-04-05
37  04/06/2020   1463     169  Kings  New York  36047.0 2020-04-06
38  04/07/2020   1395     180  Kings  New York  36047.0 2020-04-07
39  04/08/2020   1310     171  Kings  New York  36047.0 2020-04-08
40  04/09/2020   1370     188  Kings  New York  36047.0 2020-04-09
41  04/10/2020   1151     178  Kings  New York  36047.0 2020-04-10
42  04/11/2020   1129     159  Kings  New York  36047.0 2020-04-11
43  04/12/2020    702     200  Kings  New York  36047.0 2020-04-12
44  04/13/2020   1066     158  Kings  New York  36047.0 2020-04-13
45  04/14/2020   1037     152  Kings  New York  36047.0 2020-04-14
46  04/15/2020    882     132  Kings  New York  36047.0 2020-04-15
47  04/16/2020    884     126  Kings  New York  36047.0 2020-04-16
48  04/17/2020    959     118  Kings  New York  36047.0 2020-04-17
49  04/18/2020    575     116  Kings  New York  36047.0 2020-04-18

In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

fips  cases  deaths
0  36047.0   3591      96

In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

date  cases  deaths     state  county     fips         dt
416  04/20/2021    427      12  New York  Queens  36081.0 2021-04-20
417  04/21/2021    377      17  New York  Queens  36081.0 2021-04-21
418  04/22/2021    376      10  New York  Queens  36081.0 2021-04-22
419  04/23/2021    365      15  New York  Queens  36081.0 2021-04-23
420  04/24/2021    253       9  New York  Queens  36081.0 2021-04-24
421  04/25/2021    241       9  New York  Queens  36081.0 2021-04-25
422  04/26/2021    346       7  New York  Queens  36081.0 2021-04-26
423  04/27/2021    286       7  New York  Queens  36081.0 2021-04-27
424  04/28/2021    345       8  New York  Queens  36081.0 2021-04-28
425  04/29/2021    262       9  New York  Queens  36081.0 2021-04-29
426  04/30/2021    273       8  New York  Queens  36081.0 2021-04-30
427  05/01/2021    165       6  New York  Queens  36081.0 2021-05-01
428  05/02/2021    158       7  New York  Queens  36081.0 2021-05-02
429  05/03/2021    198       3  New York  Queens  36081.0 2021-05-03

In [45]:

covid.tail(50)

date    county     state     fips  cases  deaths         dt
380  03/15/2021  Richmond  New York  36085.0    225     5.0 2021-03-15
381  03/16/2021  Richmond  New York  36085.0    213     6.0 2021-03-16
382  03/17/2021  Richmond  New York  36085.0    267     4.0 2021-03-17
383  03/18/2021  Richmond  New York  36085.0    231     3.0 2021-03-18
384  03/19/2021  Richmond  New York  36085.0    267     2.0 2021-03-19
385  03/20/2021  Richmond  New York  36085.0    160     4.0 2021-03-20
386  03/21/2021  Richmond  New York  36085.0    174     3.0 2021-03-21
387  03/22/2021  Richmond  New York  36085.0    318     3.0 2021-03-22
388  03/23/2021  Richmond  New York  36085.0    252     4.0 2021-03-23
389  03/24/2021  Richmond  New York  36085.0    287     4.0 2021-03-24
390  03/25/2021  Richmond  New York  36085.0    292     3.0 2021-03-25
391  03/26/2021  Richmond  New York  36085.0    308     6.0 2021-03-26
392  03/27/2021  Richmond  New York  36085.0    214     2.0 2021-03-27
393  03/28/2021  Richmond  New York  36085.0    153     6.0 2021-03-28
394  03/29/2021  Richmond  New York  36085.0    310     1.0 2021-03-29
395  03/30/2021  Richmond  New York  36085.0    247     2.0 2021-03-30
396  03/31/2021  Richmond  New York  36085.0    261     6.0 2021-03-31
397  04/01/2021  Richmond  New York  36085.0    226     3.0 2021-04-01
398  04/02/2021  Richmond  New York  36085.0    220     2.0 2021-04-02
399  04/03/2021  Richmond  New York  36085.0    201     0.0 2021-04-03
400  04/04/2021  Richmond  New York  36085.0    142     3.0 2021-04-04
401  04/05/2021  Richmond  New York  36085.0    285     5.0 2021-04-05
402  04/06/2021  Richmond  New York  36085.0    246     4.0 2021-04-06
403  04/07/2021  Richmond  New York  36085.0    226     8.0 2021-04-07
404  04/08/2021  Richmond  New York  36085.0    234     3.0 2021-04-08
405  04/09/2021  Richmond  New York  36085.0    252     1.0 2021-04-09
406  04/10/2021  Richmond  New York  36085.0    146     3.0 2021-04-10
407  04/11/2021  Richmond  New York  36085.0    129     2.0 2021-04-11
408  04/12/2021  Richmond  New York  36085.0    199     1.0 2021-04-12
409  04/13/2021  Richmond  New York  36085.0    169     3.0 2021-04-13
410  04/14/2021  Richmond  New York  36085.0    215     1.0 2021-04-14
411  04/15/2021  Richmond  New York  36085.0    180     5.0 2021-04-15
412  04/16/2021  Richmond  New York  36085.0    184     2.0 2021-04-16
413  04/17/2021  Richmond  New York  36085.0    115     1.0 2021-04-17
414  04/18/2021  Richmond  New York  36085.0    104     2.0 2021-04-18
415  04/19/2021  Richmond  New York  36085.0    181     3.0 2021-04-19
416  04/20/2021  Richmond  New York  36085.0    144     2.0 2021-04-20
417  04/21/2021  Richmond  New York  36085.0    130     0.0 2021-04-21
418  04/22/2021  Richmond  New York  36085.0    126     3.0 2021-04-22
419  04/23/2021  Richmond  New York  36085.0    134     2.0 2021-04-23
420  04/24/2021  Richmond  New York  36085.0     77     1.0 2021-04-24
421  04/25/2021  Richmond  New York  36085.0     67     2.0 2021-04-25
422  04/26/2021  Richmond  New York  36085.0    120     1.0 2021-04-26
423  04/27/2021  Richmond  New York  36085.0     96     2.0 2021-04-27
424  04/28/2021  Richmond  New York  36085.0    111     0.0 2021-04-28
425  04/29/2021  Richmond  New York  36085.0     87     1.0 2021-04-29
426  04/30/2021  Richmond  New York  36085.0     74     3.0 2021-04-30
427  05/01/2021  Richmond  New York  36085.0     57     2.0 2021-05-01
428  05/02/2021  Richmond  New York  36085.0     58     2.0 2021-05-02
429  05/03/2021  Richmond  New York  36085.0     58     3.0 2021-05-03

In [46]:
print (data_date_dt)

2021-05-07 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

fips  deaths
1290531  1001.0   108.0
1290532  1003.0   307.0

In [48]:
Kings_death.head(2)

fips  deaths
0  36047.0    8407

In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

fips  total_deaths
1293532  51690.0          76.0
1293533  51115.0          12.0
1293534  51117.0          66.0
1293535  51119.0          20.0
1293536  51121.0          92.0
1293537  51125.0          12.0
1293538  51127.0          17.0
1293539  51700.0         224.0
1293540  51710.0         255.0
1293541  51131.0          36.0
1293542  51133.0          20.0
1293543  51720.0           7.0
1293544  51135.0          48.0
1293545  51137.0          38.0
1293546  51139.0          57.0
1293547  51141.0          43.0
1293548  51730.0          84.0
1293549  51143.0          79.0
1293550  51735.0          16.0
1293551  51740.0         190.0
1293552  51145.0          16.0
1293553  51147.0          34.0
1293554  51149.0          28.0
1293555  51153.0         485.0
1293556  51155.0          61.0
1293557  51750.0          18.0
1293558  51157.0           2.0
1293559  51159.0          15.0
1293560  51760.0         258.0
1293561  51161.0         142.0
...          ...           ...
1293752  55139.0         218.0
1293753  55141.0          93.0
1293754  56001.0          12.0
1293755  56003.0          32.0
1293756  56005.0          60.0
1293757  56007.0          23.0
1293758  56009.0          17.0
1293759  56011.0          12.0
1293760  56013.0          84.0
1293761  56015.0          24.0
1293762  56017.0           3.0
1293763  56019.0          10.0
1293764  56021.0         113.0
1293765  56023.0          12.0
1293766  56025.0         136.0
1293767  56027.0           2.0
1293768  56029.0          30.0
1293769  56031.0          11.0
1293770  56033.0          31.0
1293771  56035.0           7.0
1293772  56037.0          38.0
1293773  56039.0           9.0
1293774  56041.0          13.0
1293775  56043.0          26.0
1293776  56045.0           5.0
0        36047.0        8407.0
0        36005.0        5496.0
0        36061.0        3688.0
0        36081.0        8424.0
0        36085.0        1580.0

[250 rows x 2 columns]

In [50]:
covid_death.shape

(3251, 2)

In [51]:
print (County_covid_death)

{1001.0: 108.0, 1003.0: 307.0, 1005.0: 57.0, 1007.0: 63.0, 1009.0: 136.0, 1011.0: 41.0, 1013.0: 69.0, 1015.0: 313.0, 1017.0: 123.0, 1019.0: 45.0, 1021.0: 111.0, 1023.0: 24.0, 1025.0: 61.0, 1027.0: 56.0, 1029.0: 41.0, 1031.0: 118.0, 1033.0: 134.0, 1035.0: 29.0, 1037.0: 28.0, 1039.0: 117.0, 1041.0: 57.0, 1043.0: 193.0, 1045.0: 111.0, 1047.0: 151.0, 1049.0: 185.0, 1051.0: 205.0, 1053.0: 76.0, 1055.0: 353.0, 1057.0: 62.0, 1059.0: 82.0, 1061.0: 77.0, 1063.0: 34.0, 1065.0: 77.0, 1067.0: 44.0, 1069.0: 280.0, 1071.0: 111.0, 1073.0: 1519.0, 1075.0: 34.0, 1077.0: 241.0, 1079.0: 98.0, 1081.0: 169.0, 1083.0: 150.0, 1085.0: 53.0, 1087.0: 50.0, 1089.0: 503.0, 1091.0: 64.0, 1093.0: 100.0, 1095.0: 222.0, 1097.0: 804.0, 1099.0: 40.0, 1101.0: 585.0, 1103.0: 279.0, 1105.0: 26.0, 1107.0: 60.0, 1109.0: 77.0, 1111.0: 41.0, 1113.0: 38.0, 1117.0: 248.0, 1115.0: 239.0, 1119.0: 32.0, 1121.0: 175.0, 1123.0: 152.0, 1125.0: 451.0, 1127.0: 277.0, 1129.0: 39.0, 1131.0: 29.0, 1133.0: 72.0, 2013.0: 3.0, 2016.0: 0.0, 2

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

date     county        state    fips  cases  deaths         dt  \
1248633  2021-04-23  New Haven  Connecticut  9009.0  88101  2052.0 2021-04-23   
1251880  2021-04-24  New Haven  Connecticut  9009.0  88101  2052.0 2021-04-24   
1255127  2021-04-25  New Haven  Connecticut  9009.0  88101  2052.0 2021-04-25   
1258374  2021-04-26  New Haven  Connecticut  9009.0  88766  2059.0 2021-04-26   
1261621  2021-04-27  New Haven  Connecticut  9009.0  88949  2061.0 2021-04-27   
1264868  2021-04-28  New Haven  Connecticut  9009.0  89138  2065.0 2021-04-28   
1268115  2021-04-29  New Haven  Connecticut  9009.0  89291  2066.0 2021-04-29   
1271362  2021-04-30  New Haven  Connecticut  9009.0  89526  2069.0 2021-04-30   
1274609  2021-05-01  New Haven  Connecticut  9009.0  89526  2069.0 2021-05-01   
1277856  2021-05-02  New Haven  Connecticut  9009.0  89526  2069.0 2021-05-02   
1281103  2021-05-03  New Haven  Connecticut  9009.0  89913  2073.0 2021-05-03   
1284351  2021-05-04  New Haven  Connecticut  9009.0  90041  2077.0 2021-05-04   
1287597  2021-05-05  New Haven  Connecticut  9009.0  90248  2078.0 2021-05-05   
1290843  2021-05-06  New Haven  Connecticut  9009.0  90432  2083.0 2021-05-06   

        dt_time_delta  new_cases  
1248633      -14 days        NaN  
1251880      -13 days        0.0  
1255127      -12 days        0.0  
1258374      -11 days      665.0  
1261621      -10 days      183.0  
1264868       -9 days      189.0  
1268115       -8 days      153.0  
1271362       -7 days      235.0  
1274609       -6 days        0.0  
1277856       -5 days        0.0  
1281103       -4 days      387.0  
1284351       -3 days      128.0  
1287597       -2 days      207.0  
1290843       -1 days      184.0

In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

fips  total_cases
310  9009.0       2331.0

In [57]:
covid.tail(50)

date    county     state     fips  cases  deaths         dt
380  03/15/2021  Richmond  New York  36085.0    225     5.0 2021-03-15
381  03/16/2021  Richmond  New York  36085.0    213     6.0 2021-03-16
382  03/17/2021  Richmond  New York  36085.0    267     4.0 2021-03-17
383  03/18/2021  Richmond  New York  36085.0    231     3.0 2021-03-18
384  03/19/2021  Richmond  New York  36085.0    267     2.0 2021-03-19
385  03/20/2021  Richmond  New York  36085.0    160     4.0 2021-03-20
386  03/21/2021  Richmond  New York  36085.0    174     3.0 2021-03-21
387  03/22/2021  Richmond  New York  36085.0    318     3.0 2021-03-22
388  03/23/2021  Richmond  New York  36085.0    252     4.0 2021-03-23
389  03/24/2021  Richmond  New York  36085.0    287     4.0 2021-03-24
390  03/25/2021  Richmond  New York  36085.0    292     3.0 2021-03-25
391  03/26/2021  Richmond  New York  36085.0    308     6.0 2021-03-26
392  03/27/2021  Richmond  New York  36085.0    214     2.0 2021-03-27
393  03/28/2021  Richmond  New York  36085.0    153     6.0 2021-03-28
394  03/29/2021  Richmond  New York  36085.0    310     1.0 2021-03-29
395  03/30/2021  Richmond  New York  36085.0    247     2.0 2021-03-30
396  03/31/2021  Richmond  New York  36085.0    261     6.0 2021-03-31
397  04/01/2021  Richmond  New York  36085.0    226     3.0 2021-04-01
398  04/02/2021  Richmond  New York  36085.0    220     2.0 2021-04-02
399  04/03/2021  Richmond  New York  36085.0    201     0.0 2021-04-03
400  04/04/2021  Richmond  New York  36085.0    142     3.0 2021-04-04
401  04/05/2021  Richmond  New York  36085.0    285     5.0 2021-04-05
402  04/06/2021  Richmond  New York  36085.0    246     4.0 2021-04-06
403  04/07/2021  Richmond  New York  36085.0    226     8.0 2021-04-07
404  04/08/2021  Richmond  New York  36085.0    234     3.0 2021-04-08
405  04/09/2021  Richmond  New York  36085.0    252     1.0 2021-04-09
406  04/10/2021  Richmond  New York  36085.0    146     3.0 2021-04-10
407  04/11/2021  Richmond  New York  36085.0    129     2.0 2021-04-11
408  04/12/2021  Richmond  New York  36085.0    199     1.0 2021-04-12
409  04/13/2021  Richmond  New York  36085.0    169     3.0 2021-04-13
410  04/14/2021  Richmond  New York  36085.0    215     1.0 2021-04-14
411  04/15/2021  Richmond  New York  36085.0    180     5.0 2021-04-15
412  04/16/2021  Richmond  New York  36085.0    184     2.0 2021-04-16
413  04/17/2021  Richmond  New York  36085.0    115     1.0 2021-04-17
414  04/18/2021  Richmond  New York  36085.0    104     2.0 2021-04-18
415  04/19/2021  Richmond  New York  36085.0    181     3.0 2021-04-19
416  04/20/2021  Richmond  New York  36085.0    144     2.0 2021-04-20
417  04/21/2021  Richmond  New York  36085.0    130     0.0 2021-04-21
418  04/22/2021  Richmond  New York  36085.0    126     3.0 2021-04-22
419  04/23/2021  Richmond  New York  36085.0    134     2.0 2021-04-23
420  04/24/2021  Richmond  New York  36085.0     77     1.0 2021-04-24
421  04/25/2021  Richmond  New York  36085.0     67     2.0 2021-04-25
422  04/26/2021  Richmond  New York  36085.0    120     1.0 2021-04-26
423  04/27/2021  Richmond  New York  36085.0     96     2.0 2021-04-27
424  04/28/2021  Richmond  New York  36085.0    111     0.0 2021-04-28
425  04/29/2021  Richmond  New York  36085.0     87     1.0 2021-04-29
426  04/30/2021  Richmond  New York  36085.0     74     3.0 2021-04-30
427  05/01/2021  Richmond  New York  36085.0     57     2.0 2021-05-01
428  05/02/2021  Richmond  New York  36085.0     58     2.0 2021-05-02
429  05/03/2021  Richmond  New York  36085.0     58     3.0 2021-05-03

In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

index     fips  total_cases  E_TOTPOP   FIPS        STATE
3215  3219.0  72153.0         71.0     36439  72153  PUERTO RICO
3216     0.0      0.0          0.0       890   2060       ALASKA
3217     0.0      0.0          0.0      2132   2105       ALASKA
3218     0.0      0.0          0.0       689   2282       ALASKA
3219     0.0      0.0          0.0      1375   2164       ALASKA

In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-05-07 12:54:44.599165
5


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 108.0
1003.0 307.0
1005.0 57.0
1007.0 63.0
1009.0 136.0
1011.0 41.0
1013.0 69.0
1015.0 313.0
1017.0 123.0
1019.0 45.0
1021.0 111.0
1023.0 24.0
1025.0 61.0
1027.0 56.0
1029.0 41.0
1031.0 118.0
1033.0 134.0
1035.0 29.0
1037.0 28.0
1039.0 117.0
1041.0 57.0
1043.0 193.0
1045.0 111.0
1047.0 151.0
1049.0 185.0
1051.0 205.0
1053.0 76.0
1055.0 353.0
1057.0 62.0
1059.0 82.0
1061.0 77.0
1063.0 34.0
1065.0 77.0
1067.0 44.0
1069.0 280.0
1071.0 111.0
1073.0 1519.0
1075.0 34.0
1077.0 241.0
1079.0 98.0
1081.0 169.0
1083.0 150.0
1085.0 53.0
1087.0 50.0
1089.0 503.0
1091.0 64.0
1093.0 100.0
1095.0 222.0
1097.0 804.0
1099.0 40.0
1101.0 585.0
1103.0 279.0
1105.0 26.0
1107.0 60.0
1109.0 77.0
1111.0 41.0
1113.0 38.0
1117.0 248.0
1115.0 239.0
1119.0 32.0
1121.0 175.0
1123.0 152.0
1125.0 451.0
1127.0 277.0
1129.0 39.0
1131.0 29.0
1133.0 72.0
2013.0 3.0
2016.0 0.0
2020.0 171.0
2050.0 21.0
2997.0 0.0
2068.0 0.0
2070.0 2.0
2090.0 38.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 1.0
2150.0 5.0
2158.0 4.0
217

21203.0 22.0
21205.0 20.0
21207.0 43.0
21209.0 32.0
21211.0 70.0
21213.0 36.0
21215.0 22.0
21217.0 47.0
21219.0 25.0
21221.0 14.0
21223.0 7.0
21225.0 15.0
21227.0 170.0
21229.0 37.0
21231.0 49.0
21233.0 20.0
21235.0 55.0
21237.0 5.0
21239.0 20.0
22001.0 192.0
22003.0 91.0
22005.0 164.0
22007.0 36.0
22009.0 123.0
22011.0 77.0
22013.0 81.0
22015.0 298.0
22017.0 742.0
22019.0 429.0
22021.0 29.0
22023.0 6.0
22025.0 38.0
22027.0 55.0
22029.0 61.0
22031.0 78.0
22033.0 803.0
22035.0 25.0
22037.0 112.0
22039.0 92.0
22041.0 106.0
22043.0 56.0
22045.0 153.0
22047.0 98.0
22049.0 50.0
22051.0 894.0
22053.0 96.0
22059.0 37.0
22055.0 277.0
22057.0 204.0
22061.0 92.0
22063.0 206.0
22065.0 39.0
22067.0 75.0
22069.0 86.0
22071.0 792.0
22073.0 449.0
22075.0 27.0
22077.0 73.0
22079.0 314.0
22081.0 38.0
22083.0 56.0
22085.0 60.0
22087.0 51.0
22089.0 94.0
22091.0 10.0
22093.0 50.0
22095.0 138.0
22097.0 255.0
22099.0 116.0
22101.0 137.0
22103.0 524.0
22105.0 294.0
22107.0 8.0
22109.0 221.0
22111.0 77.0
nan 

46047.0 15.0
46049.0 13.0
46051.0 42.0
46053.0 30.0
46055.0 10.0
46057.0 39.0
46059.0 6.0
46061.0 4.0
46063.0 1.0
46065.0 39.0
46067.0 27.0
46069.0 1.0
46071.0 14.0
46073.0 16.0
46075.0 0.0
46077.0 18.0
46079.0 20.0
46081.0 47.0
46083.0 77.0
46085.0 11.0
46091.0 6.0
46087.0 25.0
46089.0 4.0
46093.0 30.0
46095.0 2.0
46097.0 9.0
46099.0 354.0
46101.0 18.0
46102.0 49.0
46103.0 190.0
46105.0 14.0
46107.0 4.0
46109.0 39.0
46111.0 3.0
46115.0 26.0
46117.0 2.0
46119.0 3.0
46121.0 30.0
46123.0 17.0
46125.0 56.0
46127.0 43.0
46129.0 16.0
46135.0 31.0
46137.0 9.0
47001.0 172.0
47003.0 128.0
47005.0 40.0
47007.0 11.0
47009.0 193.0
47011.0 149.0
47013.0 63.0
47015.0 31.0
47017.0 83.0
47019.0 158.0
47021.0 55.0
47023.0 51.0
47025.0 73.0
47027.0 31.0
47029.0 99.0
47031.0 121.0
47033.0 48.0
47035.0 132.0
47037.0 933.0
47041.0 54.0
47039.0 39.0
47043.0 110.0
47045.0 106.0
47047.0 77.0
47049.0 47.0
47051.0 90.0
47053.0 145.0
47055.0 100.0
47057.0 50.0
47059.0 155.0
47061.0 34.0
47063.0 173.0
47065.0 49

In [69]:
K_c.head()

fips  cases  deaths
0  36047.0   3591      96

In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 3591}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 44.0
1001 79.0
1009 75.0
1013 12.0
1015 82.0
1017 30.0
1031 61.0
1033 45.0
1039 27.0
1043 110.0
1045 45.0
1051 56.0
1055 84.0
1067 8.0
1069 104.0
1071 32.0
1077 103.0
1079 16.0
1083 78.0
1089 427.0
1095 139.0
1097 354.0
1103 94.0
1111 13.0
1113 67.0
1115 110.0
1117 134.0
1121 87.0
2261 14.0
4021 761.0
5009 39.0
5011 23.0
5033 26.0
5037 10.0
5045 182.0
5047 4.0
5051 47.0
5053 6.0
5063 14.0
5083 13.0
5085 86.0
5087 11.0
5115 74.0
5117 6.0
5121 21.0
5125 87.0
5131 53.0
5145 63.0
6007 251.0
6017 294.0
6023 198.0
6027 30.0
6061 504.0
6079 200.0
6089 263.0
6093 178.0
8019 8.0
8039 118.0
8047 18.0
8051 21.0
8057 1.0
8065 37.0
8067 151.0
8071 30.0
8075 42.0
8077 468.0
8083 39.0
8085 62.0
8093 46.0
8099 32.0
8101 1113.0
8103 21.0
8119 93.0
8121 9.0
9005 313.0
9007 221.0
9011 480.0
9015 219.0
10001 646.0
12019 414.0
12033 452.0
12089 166.0
12101 1537.0
12109 454.0
12113 143.0
13015 169.0
13039 29.0
13045 117.0
13047 120.0
13057 438.0
13073 207.0
13077 159.0
13085 55.0
13113 140.0
13115 75.

51045 5.0
51047 86.0
51049 26.0
51053 28.0
51057 6.0
51061 83.0
51063 22.0
51065 254.0
51067 51.0
51069 178.0
51071 37.0
51073 26.0
51075 26.0
51077 39.0
51079 31.0
51083 32.0
51085 117.0
51089 62.0
51093 51.0
51095 79.0
51097 8.0
51099 44.0
51101 41.0
51109 45.0
51113 15.0
51127 27.0
51137 63.0
51139 47.0
51143 63.0
51155 66.0
51165 101.0
51171 122.0
51177 308.0
51181 9.0
51187 95.0
51191 111.0
51193 26.0
51197 79.0
51199 73.0
51550 367.0
51580 16.0
51735 4.0
51800 123.0
51810 698.0
51820 33.0
53003 42.0
53011 1222.0
53015 267.0
53023 2.0
53027 136.0
53029 104.0
53035 499.0
53037 143.0
53041 203.0
53045 84.0
53059 41.0
53065 115.0
53067 580.0
54003 446.0
54009 54.0
54023 28.0
54025 100.0
54029 60.0
54031 52.0
54033 168.0
54035 121.0
54037 152.0
54041 31.0
54049 153.0
54051 89.0
54053 50.0
54057 52.0
54065 56.0
54067 94.0
54073 24.0
54079 221.0
54083 49.0
54085 29.0
54091 17.0
54095 26.0
54097 22.0
54103 73.0
54107 112.0
55003 39.0
55005 79.0
55009 429.0
55011 8.0
55015 57.0
55017 74.0

5067 10.0
5089 5.0
5097 1.0
5101 13.0
5111 10.0
5127 6.0
5137 3.0
5147 1.0
6003 0.0
6021 22.0
6045 54.0
6065 1058.0
6069 49.0
6073 2001.0
6095 572.0
6113 230.0
8011 36.0
8031 2163.0
8089 59.0
8105 20.0
9003 1919.0
11001 939.0
12011 7095.0
12063 75.0
12071 2831.0
12095 4524.0
12097 1634.0
12133 24.0
13001 18.0
13009 46.0
13025 13.0
13027 9.0
13033 19.0
13055 47.0
13059 114.0
13087 26.0
13089 1455.0
13091 10.0
13097 266.0
13101 5.0
13121 1724.0
13125 1.0
13151 444.0
13155 2.0
13173 11.0
13175 34.0
13189 53.0
13197 5.0
13225 52.0
13245 434.0
13249 3.0
13279 32.0
13287 28.0
13315 5.0
15009 176.0
16003 2.0
16047 6.0
16063 3.0
17003 2.0
17019 442.0
17151 7.0
17153 3.0
18001 26.0
18027 55.0
18087 76.0
18105 283.0
19021 13.0
19051 17.0
20067 1.0
20075 0.0
21011 47.0
21041 18.0
21047 108.0
21099 26.0
21115 38.0
21143 18.0
21165 22.0
21169 12.0
21171 21.0
21205 50.0
21207 31.0
21219 40.0
21235 112.0
22009 60.0
22017 366.0
22021 11.0
22025 21.0
22043 28.0
22045 138.0
22061 47.0
22067 28.0
22083 3

In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 939
36061 1162
36047 3591 3591
36081 2892
36005 1594


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 79.0, 1003.0: 260.0, 1005.0: 15.0, 1007.0: 26.0, 1009.0: 75.0, 1011.0: 9.0, 1013.0: 12.0, 1015.0: 82.0, 1017.0: 30.0, 1019.0: 11.0, 1021.0: 96.0, 1023.0: 15.0, 1025.0: 12.0, 1027.0: 42.0, 1029.0: 12.0, 1031.0: 61.0, 1033.0: 45.0, 1035.0: 5.0, 1037.0: 29.0, 1039.0: 27.0, 1041.0: 12.0, 1043.0: 110.0, 1045.0: 45.0, 1047.0: 27.0, 1049.0: 37.0, 1051.0: 56.0, 1053.0: 24.0, 1055.0: 84.0, 1057.0: 13.0, 1059.0: 33.0, 1061.0: 28.0, 1063.0: 7.0, 1065.0: 27.0, 1067.0: 8.0, 1069.0: 104.0, 1071.0: 32.0, 1073.0: 567.0, 1075.0: 19.0, 1077.0: 103.0, 1079.0: 16.0, 1081.0: 135.0, 1083.0: 78.0, 1085.0: 9.0, 1087.0: 16.0, 1089.0: 427.0, 1091.0: 11.0, 1093.0: 31.0, 1095.0: 139.0, 1097.0: 354.0, 1099.0: 26.0, 1101.0: 195.0, 1103.0: 94.0, 1105.0: 1.0, 1107.0: 8.0, 1109.0: 32.0, 1111.0: 13.0, 1113.0: 67.0, 1115.0: 110.0, 1117.0: 134.0, 1119.0: 8.0, 1121.0: 87.0, 1123.0: 57.0, 1125.0: 236.0, 1127.0: 55.0, 1129.0: 9.0, 1131.0: 8.0, 1133.0: 9.0, 2013.0: 1.0, 2016.0: 10.0, 2020.0: 443.0, 2050.0: 13.0, 206

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 954438
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 76.0 38921 195.26733639937308
1001 108.0 55869 193.3093486548891
1009 136.0 57826 235.18832359146407
1013 69.0 19448 354.79226655697244
1015 313.0 113605 275.51604242771003
1017 123.0 33254 369.88031515005713
1031 118.0 52342 225.44037293187117
1033 134.0 55241 242.5734508788762
1039 117.0 37049 315.7979972468893
1043 193.0 83768 230.39824276573393
1045 111.0 49172 225.738225006101
1051 205.0 81209 252.43507493011856
1055 353.0 102268 345.1715101498025
1067 44.0 17205 255.73961057832025
1069 280.0 105882 264.4453259288642
1071 111.0 51626 215.00794173478482
1077 241.0 92729 259.8971195634591
1079 98.0 32924 297.65520592880574
1083 150.0 98915 151.64535206995905
1089 503.0 372909 134.88545462834097
1095 222.0 96774 229.40045880091762
1097 804.0 413210 194.5741874591612
1103 279.0 119679 233.12360564510067
1111 41.0 22722 180.4418625121028
1113 38.0 57961 65.56132571901796
1115 239.0 89512 267.0033068191974
1117 248.0 217702 113.91718955269128
1121 175.0 79978 218.81017279751933
22

26147 406.0 159128 255.14051581117087
26149 94.0 60964 154.18935765369727
26151 107.0 41170 259.8979839689094
26155 99.0 68122 145.32750066057955
26157 164.0 52245 313.90563690305294
26159 119.0 75677 157.24724817315698
26165 42.0 33631 124.88477892420684
27003 433.0 356921 121.31536110231676
27005 50.0 34423 145.2517212328966
27015 40.0 25008 159.94881637875878
27017 53.0 35871 147.7516656909481
27023 38.0 11800 322.03389830508473
27025 51.0 56579 90.13945103306881
27029 16.0 8818 181.44704014515764
27033 21.0 11196 187.56698821007504
27035 88.0 65055 135.2701560218277
27041 74.0 38141 194.01693715424346
27043 19.0 13653 139.16355379770016
27045 10.0 21067 47.46760336070632
27047 29.0 30281 95.76962451702387
27049 72.0 46340 155.37332757876564
27051 8.0 5972 133.95847287340925
27055 15.0 18600 80.64516129032258
27059 62.0 40596 152.72440634545276
27061 53.0 45130 117.43851096831376
27063 11.0 9846 111.72049563274427
27065 24.0 16337 146.90579665789312
27067 82.0 43199 189.819208777980

51047 61.0 52605 115.95855907233152
51049 9.0 9932 90.61619009262988
51053 43.0 28544 150.64461883408072
51057 9.0 10953 82.16926869350863
51061 66.0 71222 92.66799584398079
51063 21.0 15749 133.34179947933202
51065 18.0 27270 66.00660066006601
51067 76.0 56042 135.61257628207417
51069 100.0 89313 111.96578325663677
51071 21.0 16720 125.59808612440192
51073 48.0 37348 128.52093820284887
51075 23.0 23753 96.82987412116364
51077 35.0 15550 225.08038585209002
51079 18.0 19819 90.82193854382159
51083 77.0 33911 227.06496417091796
51085 155.0 107766 143.830150511293
51089 121.0 50557 239.3338212314813
51093 67.0 37109 180.54919291816
51095 72.0 76523 94.08935875488416
51097 8.0 7025 113.87900355871885
51099 20.0 26836 74.52675510508273
51101 11.0 17148 64.14742243993469
51109 32.0 37591 85.12675906466973
51113 16.0 13261 120.65455093884323
51127 17.0 23091 73.62175739465593
51137 38.0 37051 102.56133437693988
51139 57.0 23902 238.47376788553257
51143 79.0 60354 130.89438976704113
51155 61.0

13149 19.0 11923 159.35586681204396
13159 38.0 14219 267.2480483859624
13171 57.0 19077 298.7891177858154
13177 58.0 29992 193.38490264070418
13181 26.0 7921 328.2413836636788
13185 210.0 117406 178.86649745328177
13195 56.0 29880 187.41633199464525
13201 13.0 5718 227.35222105631337
13219 72.0 40280 178.7487586891758
13221 29.0 15259 190.05177272429387
13223 214.0 168667 126.87721961023792
13237 56.0 22119 253.17600253176002
13241 49.0 17137 285.93102643403165
13255 188.0 66703 281.8463937154251
13257 82.0 25925 316.2970106075217
13275 123.0 44451 276.7091853951542
13281 56.0 12037 465.2322007144638
13285 225.0 69922 321.78713423529075
13291 73.0 24511 297.8254661172535
13293 126.0 26320 478.72340425531917
13305 88.0 29927 294.0488522070371
13317 27.0 9777 276.15833077631174
16005 106.0 87808 120.71793002915452
16007 2.0 6125 32.6530612244898
16009 9.0 9298 96.79500967950096
16011 74.0 46811 158.08250197603127
16013 18.0 23021 78.18947917119152
16015 2.0 7831 25.539522410930914
16019 

38029 13.0 3241 401.11076828139466
38035 77.0 69451 110.86953391599833
38037 7.0 2274 307.82761653474057
38039 2.0 2231 89.64589870013447
38045 17.0 4046 420.16806722689074
38047 9.0 1850 486.48648648648646
38051 7.0 2497 280.3364036844213
38063 15.0 2879 521.0142410559222
38071 30.0 11519 260.43927424255577
38077 17.0 16177 105.08746986462262
38083 2.0 1315 152.0912547528517
38087 0.0 750 0.0
38089 51.0 31489 161.96131982597097
38093 81.0 20704 391.2287480680062
38101 194.0 67641 286.8082967430996
38103 8.0 3834 208.65936358894103
39001 52.0 27698 187.73918694490575
39031 58.0 36600 158.46994535519124
39041 131.0 209177 62.62638817843262
39047 49.0 28525 171.7791411042945
39049 1392.0 1316756 105.71434646965724
39079 51.0 32413 157.34427544503748
39087 102.0 59463 171.53524040159428
39097 61.0 44731 136.37074959200555
39101 135.0 65093 207.39557248859322
39115 23.0 14508 158.53322304935207
39129 121.0 58457 206.98975315188943
39131 33.0 27772 118.8247155408325
39173 188.0 130817 143.7

53053 655.0 904980 72.37729010585869
53055 0.0 17582 0.0
53061 570.0 822083 69.33606460661515
53063 633.0 522798 121.07926962230155
53071 64.0 60760 105.3324555628703
53073 92.0 229247 40.131386670272676
54001 26.0 16441 158.1412322851408
54007 3.0 13957 21.49459052805044
54011 171.0 91945 185.98074936103106
54017 6.0 8448 71.02272727272728
54019 75.0 42406 176.86176484459745
54027 34.0 23175 146.70981661272924
54039 337.0 178124 189.19404459814513
54061 83.0 105612 78.5895542173238
54063 17.0 13275 128.060263653484
54069 83.0 41411 200.42983748279443
54071 9.0 6969 129.14334911752044
54077 46.0 33432 137.59272553242403
54081 83.0 73361 113.13913387222094
54093 10.0 6839 146.22020763269484
55001 17.0 20220 84.0751730959446
55007 19.0 15036 126.36339451981911
55013 22.0 15414 142.72739068379394
55019 62.0 34774 178.29412779662965
55025 325.0 546695 59.44813835868263
55029 28.0 27668 101.19994217146161
55033 32.0 45368 70.53429730206312
55037 12.0 4295 279.39464493597205
55051 40.0 5687 

28017 57.0 17103 333.274864058937
28025 54.0 19316 279.5609857113274
28035 152.0 74897 202.94537831955887
28047 309.0 208080 148.50057670126876
28049 414.0 231840 178.57142857142856
28061 48.0 16383 292.9866324848929
28101 61.0 21018 290.22742411266535
28105 98.0 49587 197.63244398733542
28127 88.0 26658 330.107284867582
28141 67.0 19383 345.66372594541605
28153 41.0 20183 203.1412574939305
29055 37.0 23920 154.68227424749165
29085 15.0 9544 157.16680637049456
29143 43.0 17076 251.81541344577184
29153 18.0 9174 196.20667102681492
29181 16.0 13288 120.40939193257074
29203 12.0 8166 146.9507714915503
30051 1.0 2337 42.78990158322636
31079 117.0 61353 190.69972128502275
31091 2.0 682 293.2551319648094
32003 4323.0 2266715 190.7165214859389
32021 7.0 4505 155.3829078801332
34007 1277.0 506471 252.13684495262314
34021 936.0 367430 254.74239991290858
34023 2314.0 825062 280.46377120749713
35003 5.0 3527 141.7635384179189
35007 48.0 11941 401.9763838874466
35009 78.0 48954 159.33325162397352


48495 21.0 8010 262.17228464419475
49039 22.0 30939 71.1076634668218
51029 30.0 17148 174.94751574527643
51081 25.0 11336 220.5363443895554
51153 485.0 470335 103.11799036856709
51590 135.0 40044 337.1291579262811
51620 32.0 7967 401.65683444207355
51660 95.0 53016 179.19118756601782
51683 47.0 41085 114.3969818668614
51685 11.0 17478 62.9362627302895
51690 76.0 12554 605.3847379321331
51700 224.0 179225 124.98256381643186
51740 190.0 94398 201.27545075107525
51760 258.0 230436 111.96167265531426
53025 119.0 97733 121.7603061401983
54013 0.0 7109 0.0
54109 39.0 20394 191.23271550456016
1065 77.0 14651 525.5613951266125
2013 3.0 3337 89.90110878034163
2185 4.0 9832 40.68348250610252
4009 79.0 38837 203.41426989726295
4012 80.0 21108 379.00322152738295
5035 96.0 47955 200.18767594619956
5095 17.0 6701 253.6934785852858
5099 24.0 8252 290.838584585555
6039 242.0 157327 153.81975121879907
6071 4677.0 2180085 214.53291958799772
6075 543.0 881549 61.596122280213585
6077 1391.0 762148 182.510

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(2083.0, 854757, 243.69499167599682)

In [91]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 6863.711353871355,
 'ALABAMA': 18103.24564966316,
 'ALASKA': 760.6943126879594,
 'ARIZONA': 4599.499517204787,
 'ARKANSAS': 16796.805966587952,
 'CALIFORNIA': 5972.298583800309,
 'COLORADO': 7555.079166686133,
 'CONNECTICUT': 1584.408828373442,
 'DELAWARE': 537.2807096294907,
 'FLORIDA': 12835.37451815298,
 'GEORGIA': 42707.692878586306,
 'IDAHO': 4971.758207336921,
 'ILLINOIS': 23001.91572312068,
 'INDIANA': 19413.552329078666,
 'IOWA': 24613.594780981923,
 'KANSAS': 27144.847617476516,
 'KENTUCKY': 20716.19003994533,
 'LOUISIANA': 17090.33092381585,
 'MAINE': 827.9631189786714,
 'MARYLAND': 3480.950190276856,
 'MASSACHUSETTS': 2954.250538290081,
 'MICHIGAN': 14834.175287495083,
 'MINNESOTA': 12461.395804471516,
 'MISSISSIPPI': 24157.90119454126,
 'MISSOURI': 17225.68595486333,
 'MONTANA': 11416.640786306782,
 'NEBRASKA': 13782.452975810462,
 'NEVADA': 2160.176571908025,
 'NEW HAMPSHIRE': 807.3599290138142,
 'NEW JERSEY': 5639.673723115438,
 'NEW YORK': 9096.34240161986

In [92]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [93]:
Covid_death_capita[72049]

nan

In [94]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [95]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 954437.9999999999, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [96]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [97]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76274734.0 1000000 10122.644282181305
ALABAMA 954437.9999999999 76274734.0 1000000 12513.160648977155
ALASKA 231145.0 76274734.0 1000000 3030.4268252184265
ARIZONA 1839932.0 76274734.0 1000000 24122.43089566199
ARKANSAS 830467.0000000003 76274734.0 1000000 10887.838691118874
CALIFORNIA 11847711.0 76274734.0 1000000 155329.43058182282
COLORADO 1337805.0 76274734.0 1000000 17539.294204552716
CONNECTICUT 874974.0 76274734.0 1000000 11471.347772907344
DELAWARE 239009.0 76274734.0 1000000 3133.527807517493
FLORIDA 3892551.999999999 76274734.0 1000000 51033.30809387023
GEORGIA 1928702.999999999 76274734.0 1000000 25286.263207420678
IDAHO 340742.0000000001 76274734.0 1000000 4467.298437251845
ILLINOIS 2987496.0000000005 76274734.0 1000000 39167.5702205661
INDIANA 1602975.9999999995 76274734.0 1000000 21015.818947333195
IOWA 699741.0 76274734.0 1000000 9173.955296913917
KANSAS 401103.0000000001 76274734.0 1000000 5258.661406803465
KENTUCKY 1416013.0000000002 76274734.0 1000

In [98]:
print (Covid_state)

{'NEW MEXICO': 2845.0, 'ALABAMA': 4644.0, 'ALASKA': 1710.0, 'ARIZONA': 9286.0, 'ARKANSAS': 2493.0, 'CALIFORNIA': 24408.0, 'COLORADO': 20265.0, 'CONNECTICUT': 7492.0, 'DELAWARE': 2971.0, 'FLORIDA': 62140.0, 'GEORGIA': 15521.0, 'IDAHO': 2380.0, 'ILLINOIS': 32221.0, 'INDIANA': 13773.0, 'IOWA': 4652.0, 'KANSAS': 2677.0, 'KENTUCKY': 7402.0, 'LOUISIANA': 5962.0, 'MAINE': 3939.0, 'MARYLAND': 10117.0, 'MASSACHUSETTS': 12962.0, 'MICHIGAN': 49161.0, 'MINNESOTA': 19656.0, 'MISSISSIPPI': 2743.0, 'MISSOURI': 7421.0, 'MONTANA': 1723.0, 'NEBRASKA': 2614.0, 'NEVADA': 4642.0, 'NEW HAMPSHIRE': 2984.0, 'NEW JERSEY': 21640.0, 'NEW YORK': 33630.0, 'NORTH CAROLINA': 22318.0, 'NORTH DAKOTA': 1689.0, 'OHIO': 18221.0, 'OKLAHOMA': 3248.0, 'OREGON': 10038.0, 'PENNSYLVANIA': 39651.0, 'RHODE ISLAND': 3762.0, 'SOUTH CAROLINA': 9289.0, 'SOUTH DAKOTA': 1323.0, 'TENNESSEE': 10646.0, 'TEXAS': 39293.0, 'UTAH': 4423.0, 'VERMONT': 875.0, 'VIRGINIA': 12218.0, 'WASHINGTON': 17667.0, 'WEST VIRGINIA': 4838.0, 'WISCONSIN': 915

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [99]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [100]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [101]:
Total_covid_death_cap


{'NEW MEXICO': 6863.711353871355,
 'ALABAMA': 18103.24564966316,
 'ALASKA': 760.6943126879594,
 'ARIZONA': 4599.499517204787,
 'ARKANSAS': 16796.805966587952,
 'CALIFORNIA': 5972.298583800309,
 'COLORADO': 7555.079166686133,
 'CONNECTICUT': 1584.408828373442,
 'DELAWARE': 537.2807096294907,
 'FLORIDA': 12835.37451815298,
 'GEORGIA': 42707.692878586306,
 'IDAHO': 4971.758207336921,
 'ILLINOIS': 23001.91572312068,
 'INDIANA': 19413.552329078666,
 'IOWA': 24613.594780981923,
 'KANSAS': 27144.847617476516,
 'KENTUCKY': 20716.19003994533,
 'LOUISIANA': 17090.33092381585,
 'MAINE': 827.9631189786714,
 'MARYLAND': 3480.950190276856,
 'MASSACHUSETTS': 2954.250538290081,
 'MICHIGAN': 14834.175287495083,
 'MINNESOTA': 12461.395804471516,
 'MISSISSIPPI': 24157.90119454126,
 'MISSOURI': 17225.68595486333,
 'MONTANA': 11416.640786306782,
 'NEBRASKA': 13782.452975810462,
 'NEVADA': 2160.176571908025,
 'NEW HAMPSHIRE': 807.3599290138142,
 'NEW JERSEY': 5639.673723115438,
 'NEW YORK': 9096.34240161986

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [102]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [103]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [104]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [105]:
#print (Medicaid_demand)

In [106]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [107]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [108]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [109]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)